In [ ]:
import numpy as np
import cv2
from imutils import paths
import imutils
import math
import collections as coll

import os
from pyspin import PySpin
import matplotlib.pyplot as plt
import sys
import keyboard
import time

from PIL import Image




def GetBlurredGrayScaleImage(imagePath):
    img = cv2.imread(imagePath)
    #img = ~img
    #img = cv2.imread('TriangleWithDot.jpg')
    img = cv2.resize(img, (800, 800))
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgGray = cv2.GaussianBlur(imgGray, (5, 5), 0)
    return imgGray

def GetBlurredGrayScaleImageFromImage(img):
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgGray = cv2.GaussianBlur(imgGray, (5, 5), 0)
    return imgGray


def DrawContours(contours, img):
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue
        
        cv2.drawContours(img, [approx], 0, (0, 0, 255), 3)
        
def DrawContours(contours, img, color):
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue
        
        cv2.drawContours(img, [approx], 0, color, 3)
        
def DrawContoursWithText(contours, img, text):
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue
        
        cv2.drawContours(img, [approx], 0, (0, 0, 255), 3)
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))

def DrawContoursWithTextInCenter(contourIDs, contours, img, text, color):
    for contourID in contourIDs:
        contour = contours[contourID]
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue
        
        cv2.drawContours(img, [approx], 0, color, 3)
        
        # compute center of contour
        M = cv2.moments(contour)
        area = float(M["m00"])

        cX = 0
        cY = 0
        if area > 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
        cv2.putText(img, text, (cX, cY), cv2.FONT_HERSHEY_COMPLEX, 0.5, color)
        
def DrawContoursFromIDWithText(contourIDs, contours, img, text):
      
    for contourID in contourIDs:
        approx = cv2.approxPolyDP(contours[contourID], 0.01*cv2.arcLength(contours[contourID], True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue
        
        cv2.drawContours(img, [approx], 0, (0, 0, 255), 3)
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))

        
def DrawContoursFromIDWithTextInCenter(contourIDs, contours, img, text):
    if(len(contourIDs) > 0):
        allX = 0
        allY = 0

        for contourID in contourIDs:
            approx = cv2.approxPolyDP(contours[contourID], 0.01*cv2.arcLength(contours[contourID], True), True)

            # if too few curves skip this contour
            if len(approx) < 3:
                continue

            cv2.drawContours(img, [approx], 0, (0, 0, 255), 3)
            x = approx.ravel()[0]
            y = approx.ravel()[1]
            allX = allX + x
            allY = allY + y

        x = int(allX / len(contourIDs))
        y = int(allY / len(contourIDs))
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))
    
    

def DrawContoursFromID(contourIDs, contours, img, color):
    for contourID in contourIDs:
        approx = cv2.approxPolyDP(contours[contourID], 0.01*cv2.arcLength(contours[contourID], True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue
        
        cv2.drawContours(img, [approx], 0, color, 3)
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        cv2.putText(img, str(contourID), (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, color)

        
def DrawGroupedContoursFromID(contourIDsList, contours, img):
    for group in contourIDsList:
        DrawContoursFromIDWithTextInCenter(group, contours, img, str(len(group)))


def DrawContoursWithArea(contours, hierarchy, img, minArea, maxArea):
    
    for component in zip(contours, hierarchy):
        contour = component[0]
        conHier = component[1]
        #approximate the number of polygonal curves
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue

        # compute center of contour
        M = cv2.moments(contour)
        area = float(M["m00"])

        if area < minArea:
            continue
        if area > maxArea:
            continue
        else:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            
        
        # drawContours(imageToDrawOnto, contourOrApproximate, 
        #         indexOfContour (here always 0 since we are iterating over the contoursvia for), lineColor, lineThickness )
        cv2.drawContours(img, [approx], 0, (0, 0, 255), 3)
        x = approx.ravel()[0]
        y = approx.ravel()[1]

        # this gives inner most children since there are no child nodes
        if conHier[2] < 0:
            cv2.putText(img, str(len(conHier)), (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))
            
        # this gives outer most parent since there are no child nodes
        if conHier[3] < 0:
            cv2.putText(img, str(0), (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))
        
        #cv2.putText(img, conHier[2], (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))
        # if we have three polygonal curves it is triangle
#         if len(approx) == 3:
#             cv2.putText(img, "Triangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))
#         # Square
#         if len(approx) == 4:
#     #         # for checking if it square or rectangle
#     #         x, y, w, h = cv2.boundingRect(approx)
#     #         aspectRatio = float(w)/h
#     #         print(aspectRatio)
#     #         if aspectRatio >= 0.95 and aspectRatio <= 1.05:

#             cv2.putText(img, "Square", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))
#         # Pentagon
#         if len(approx) == 5:
#             cv2.putText(img, "Pentagon", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))

            

            
def getContoursWithArea(contours, hierarchy, minArea, maxArea):
    returnContours = []
    returnHierarchy = []
    
    for component in zip(contours, hierarchy):
        contour = component[0]
        conHier = component[1]
        
        #approximate the number of polygonal curves
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue

        # compute center of contour
        M = cv2.moments(contour)
        area = float(M["m00"])

        if area == 0:
            continue
        elif area < minArea:
            continue
        elif area > maxArea:
            continue
        else:
            returnContours.append(contour)
            returnHierarchy.append(conHier)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
    return returnContours, returnHierarchy

def getContourIDsWithArea(contours, hierarchy, minArea, maxArea):
    returnContours = []
    returnHierarchy = []
    
    i = -1
    for component in zip(contours, hierarchy):
        i += 1
        contour = component[0]
        conHier = component[1]
        
        #approximate the number of polygonal curves
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue

        # compute center of contour
        M = cv2.moments(contour)
        area = float(M["m00"])

        if area == 0:
            continue
        elif area < minArea:
            continue
        elif area > maxArea:
            continue
        else:
            returnContours.append(i)
            returnHierarchy.append(conHier)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
        
    return returnContours, returnHierarchy


def getDotsInOutline(outlineID, contours, hierarchy, minAreaDot, maxAreaDot):
    #print("Outline ID ", outlineID)
    
    # this returns a list of size contours where every contour id that is not a dot will be set to -1
    dotIDs = getDotIDsInOutline(outlineID, contours, hierarchy, minAreaDot, maxAreaDot)
    print("IDs: ", dotIDs )
    returnDotCon = []
    for i in range(0, len(dotIDs)):
        if dotIDs[i] > 0:
            returnDotCon.append(contours[dotIDs[i]])
    
    return returnDotCon



def getDotIDsInOutlineWithEccentricity(contourID, contours, hierarchy, minDotSize, maxDotSize, dotMinEcc, dotMaxEcc):
    
    returnContours = []
    skippedOutlineContourIDs = []
    
    if contourID > len(contours) or contourID > len(hierarchy):
        #print("ID is bigger than contours. -1 will be returned")
        #returnContours.append(-1)
        return returnContours, skippedOutlineContourIDs
    
    # get child contour of outline
    # return outline contour if there is no child contour -> make this recursive
    if hierarchy[contourID][2] > 0:
        
        skippedOutlineContourIDs.append(contourID)
        # get all children IDs
        childContourIDs = getAllContourIDsOfSameHierarchy(hierarchy[contourID][2], contours, hierarchy)
        #print("ChildContourIDS ", childContourIDs)
        
        # add children ids to return ids and return them
        for i in range(0, len(childContourIDs)):
            tmpRet, tmpOut = getDotIDsInOutlineWithEccentricity(childContourIDs[i], contours, hierarchy, minDotSize, maxDotSize, dotMinEcc, dotMaxEcc)
            returnContours.extend(tmpRet)
            skippedOutlineContourIDs.extend(tmpOut)
        return returnContours, skippedOutlineContourIDs
        
    else:
        # compute area of contour
        M = cv2.moments(contours[contourID])
        area = float(M["m00"])

        # return empty if area is too big or small for dot
        # return outline contour otherwise
        if area > maxDotSize:
            #print("Area is bigger")
            #returnContours.append(-1)
            return returnContours, skippedOutlineContourIDs
        elif area < minDotSize:
            #print("Area is smaller")
            #returnContours.append(-1)
            return returnContours, skippedOutlineContourIDs
        else:
            # need at least 5 points for ellipse
            if len(contours[contourID]) < 5:
                return returnContours, skippedOutlineContourIDs
            ellipse = cv2.fitEllipse(contours[contourID])
            width = ellipse[1][0]
            height = ellipse[1][1]
            ecc = 0
            if width < height:
                ecc = math.sqrt(1 - width**2 / height**2)
            else:
                ecc = math.sqrt(1 - height**2 / width**2)
            
            if ecc < dotMinEcc or ecc > dotMaxEcc:
                return returnContours, skippedOutlineContourIDs
            else:
                returnContours.append(contourID)
                return returnContours, skippedOutlineContourIDs



def getDotIDsInOutline(contourID, contours, hierarchy, minDotSize, maxDotSize):
    returnContours = []
    skippedOutlineContourIDs = []
    
    if contourID > len(contours) or contourID > len(hierarchy):
        #print("ID is bigger than contours. -1 will be returned")
        #returnContours.append(-1)
        return returnContours, skippedOutlineContourIDs
    
    # get child contour of outline
    # return outline contour if there is no child contour -> make this recursive
    if hierarchy[contourID][2] > 0:
        
        skippedOutlineContourIDs.append(contourID)
        # get all children IDs
        childContourIDs = getAllContourIDsOfSameHierarchy(hierarchy[contourID][2], contours, hierarchy)
        #print("ChildContourIDS ", childContourIDs)
        
        # add children ids to return ids and return them
        for i in range(0, len(childContourIDs)):
            tmpRet, tmpOut = getDotIDsInOutline(childContourIDs[i], contours, hierarchy, minDotSize, maxDotSize)
            returnContours.extend(tmpRet)
            skippedOutlineContourIDs.extend(tmpOut)
        return returnContours, skippedOutlineContourIDs
        
    else:
        # compute area of contour
        M = cv2.moments(contours[contourID])
        area = float(M["m00"])

        # return empty if area is too big or small for dot
        # return outline contour otherwise
        if area > maxDotSize:
            #print("Area is bigger")
            #returnContours.append(-1)
            return returnContours, skippedOutlineContourIDs
        elif area < minDotSize:
            #print("Area is smaller")
            #returnContours.append(-1)
            return returnContours, skippedOutlineContourIDs
        else:
            #print("Should return 0 contour")
            returnContours.append(contourID)
            return returnContours, skippedOutlineContourIDs



        
        
        
        
        
        
        
# this should return an array which contains subarrays which are the dots close to each other
# Example Form: [[0,1,2],[3,4],[5,8,9,10],[7]]
def getGroupedDotsIDs(contourIDs, contours, hierarchy, maxDist):
    # control that there are enough entries
    if len(contourIDs) < 2:
        return [contourIDs]
    
    # copy contourIDs into seperate list which will then be emptied over time
    IDs = contourIDs.copy()
    returnIDs = []
    
    
    # this is to keep track if all entires have been sorted into buckets
    addedIDs = []
    #while len(addedIDs) < len(contourIDs):
    while IDs:
        if len(IDs) < 1:
            break
        
        visited = []
        queue = []

        # get all contours whose center is within maxDIst
        queue = getIDsOfMaxDistance(IDs[0], IDs, contours, maxDist)
        
        # stop if there are no contours
        if len(queue) < 1:
            break

        
        # while there are still contours to check the distance
        while queue:
            # get first entry and add it to the visited list
            # then control for next one if it also has close nodes in list
            p = queue.pop(0)
            visited.append(p)
            addedIDs.append(p)
            
            # remove popped entry from IDs so that it wont be visited again
            if p in IDs:
                IDs.remove(p)
            
            # remove other closest entries from IDs since they are already in the queue
            for deleteID in queue:
                if deleteID in IDs:
                    IDs.remove(deleteID)
                
            if len(queue) > 0:
                queue.extend(getIDsOfMaxDistance(queue[0], IDs, contours, maxDist))
            
        # set(x) removes double entries 
        visited = list(set(visited))
        returnIDs.append(visited)
    
    
        
    return returnIDs
        
    
    # redo until list is empty

def getIDsOfMaxDistance(currentID, IDs, contours, maxDist):
    returnIDs = []
    
    currentCon = contours[currentID]
    
    # compute the center of the contour
    M = cv2.moments(currentCon)
    
    # float(M["m00"]) is area
    if float(M["m00"]) > 0:
        cX = float(M["m10"] / M["m00"])
        cY = float(M["m01"] / M["m00"])

        for ID in IDs:
            idContour = contours[ID]
            # compute the center of the contour
            idM = cv2.moments(idContour)
            if float(idM["m00"]) > 0:
                idCX = int(idM["m10"] / idM["m00"])
                idCY = int(idM["m01"] / idM["m00"])

                dist = getDistance(cX, cY, idCX, idCY)
                #print("Compared: ", str(currentCon), str(ID))

                if dist > maxDist:
                    continue
                else:
                    #print("Added: ", str(ID))
                    returnIDs.append(ID)
        
      
    return returnIDs

            
            
            
def getDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    #print("Dist ", str(dist))
    return dist

def getDistanceOfIDs(id1, id2, contours):
    con1 = contours[id1]
    con2 = contours[id2]
    
    # compute the center of the contour
    M1 = cv2.moments(con1)
    M2 = cv2.moments(con2)
    
    # float(M["m00"]) is area
    if float(M1["m00"]) > 0 and float(M2["m00"]) > 0:
        cX1 = float(M1["m10"] / M1["m00"])
        cY1 = float(M1["m01"] / M1["m00"])
        
        cX2 = float(M2["m10"] / M2["m00"])
        cY2 = float(M2["m01"] / M2["m00"])
        
        dist = getDistance(cX1, cY1, cX2, cY2)
        return dist
    
    return -1
    
def getAllContourIDsOfSameHierarchy(contourID, contours, hierarchies):
    returnContourIDs = []
    
    # append the given contour
    returnContourIDs.append(contourID)
    
    # get the next contour from the given
    nextContourID = contourID
    
    # while there is a next contour, get next and add the one to returnContours
    while hierarchies[nextContourID][0] > 0:
        nextContourID = hierarchies[nextContourID][0]
        
        returnContourIDs.append(nextContourID)

    return returnContourIDs  
    

def getAllContoursOfSameHierarchy(contourID, contours, hierarchies):
    returnContours = []
    returnHierarchies = []
    
    # append the given contour
    returnContours.append(contours[contourID])
    returnHierarchies.append(hierarchies[contourID])
    
    # get the next contour from the given
    nextContourID = contourID
    
    # while there is a next contour, get next and add the one to returnContours
    while hierarchies[nextContourID][0] > 0:
        nextContourID = hierarchies[nextContourID][0]
        
        returnContours.append(contours[nextContourID])
        returnHierarchies.append(hierarchies[nextContourID])

    return returnContours, returnHierarchies
        


# this returns only child Contours, so contours that have no other contour in them
def getChildContours(contours, hierarchy):
    returnContours = []
    returnHierarchy = []
    
    for component in zip(contours, hierarchy):
        contour = component[0]
        conHier = component[1]
        #approximate the number of polygonal curves
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue

        # this gives inner most children since there are no child nodes
        if conHier[2] < 0:
            returnContours.append(contour)
            returnHierarchy.append(conHier)
    
    return returnContours, returnHierarchy
            

            
            
def trackChanged(x):
    pass
            
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result           
            

## this should return the ID and the image rotation from the array that contains the counts of dots
## 
def getTileIDAndShiftsFromGroupedArray(array, library):
    # this return -1 i the array is not found in the library
    ID = getArrayID(array, library)
    shifts = -1
    
    if ID >= 0:
        # library is in form: [[id0, [x,x,x,x,x,x,x,x]], [id1,[y,y,y,y,y,y,y,y]], ...]
        # the id hereby should be the index of the entry
        shifts = getShiftsOfArray(array, library[ID][1])
        # return degree from shift count and ID
#         if(shifts >= 0):
#             rotation = getRotationFromShifts(shifts)
            


    return ID, shifts
    
    
    # later: compare length of array against library to determine if something is occluded

    
def getSimpleRotationFromShifts(shifts):
    return shifts * 45    

def getZRotationFromShifts(shifts, centerArray, outlineID, contours):
    
    if shifts < 0:
        return -1
    if shifts > len(centerArray):
        return -1
    
    # get the centers coordinates
    outlineCenter = getCenterCoordinates([outlineID], contours)
    
    # for now without form
    # might be able without form since we will just compare the element that should be first
    zeroEntryCenter = centerArray[(len(centerArray) - shifts) % len(centerArray)]
    
    centerToZeroVec = [zeroEntryCenter[0] - outlineCenter[0], zeroEntryCenter[1] - outlineCenter[1]]
    print("zero angle ", getAngle(centerToZeroVec))
    return getAngle(centerToZeroVec)

def getXYRotationFromShiftsForSquare(shifts, centerArray):
    # Y rotation should be the same just for other corners comparison
    
    # get corners and compare front and back distance
#     5----6----7
#     |         |
#     4         0
#     |         |
#     3----2----1
    # compare diff(1,3) against diff(5, 7) for x rotation
    # compare diff(1,7) against diff(5, 3) for y rotation
    if len(centerArray) > 0:
        corner1 = centerArray[(len(centerArray) - shifts + 1) % len(centerArray)]
        corner3 = centerArray[(len(centerArray) - shifts + 3) % len(centerArray)]
        corner5 = centerArray[(len(centerArray) - shifts + 5) % len(centerArray)]
        corner7 = centerArray[(len(centerArray) - shifts + 7) % len(centerArray)]
        point0 = centerArray[(len(centerArray) - shifts + 0) % len(centerArray)]
        point2 = centerArray[(len(centerArray) - shifts + 2) % len(centerArray)]
        point4 = centerArray[(len(centerArray) - shifts + 4) % len(centerArray)]
        point6 = centerArray[(len(centerArray) - shifts + 6) % len(centerArray)]

        diff13 = getDiffOfPoints(corner1, corner3)
        diff57 = getDiffOfPoints(corner5, corner7)
        diff17 = getDiffOfPoints(corner1, corner7)
        diff35 = getDiffOfPoints(corner3, corner5)
        width = getDiffOfPoints(point0, point4)
        height = getDiffOfPoints(point2, point6)

        xRot = getRotationFromDifference(diff13, diff57)
        yRot = getRotationFromDifference(diff17, diff35)
        return xRot, yRot
    else:
        return 0, 0
    
def getRotationFromDifference(len1, len2):
    print("Need to test with rotated picture")
    if len1 > 0 and len2 > 0:
        ratio = len1 / len2
        print("ratio ", ratio)
        #return mapRange(ratio, 0, 2, -90, 90)
        if ratio > 1:
            # should be pos
            return mapRange(ratio, 1, 5, 0, 90)
        elif ratio < 1:
            # should be negative
            # will be between 0 and 1
            # needs to be between 0 and -90
            return mapRange(ratio, 0, 1, -90, 0)
        else:
            return 0
        #GK = len1 * 0.5 - len2 * 0.5
        #sin = GK / hypo
    # should get the angle depending on the difference of the two lines/lengths of the lines
    # need to keep track of positive and negative rotation
    # maybe start by dividing the two
    # if it is almost 1 -> rotation = 0
    # the more different -> rotation = XXX

    
def mapRange(x, valueMin, valueMax, targetMin, targetMax):
    if x > valueMax:
        x = valueMax
    if x < valueMin:
        x = valueMin
    
    x = x - valueMin
    x = x / (valueMax - valueMin)
    x = x * (targetMax - targetMin)
    x = x + targetMin
    return x
    
    
def getDiffOfPoints(point1, point2):
    diffVec = np.array([point1[0] - point2[0], point1[0] - point2[0]])
    return np.linalg.norm(diffVec)

def getArrayID(array, library):
    
    # library is in form: [[id0, [x,x,x,x,x,x,x,x]], [id1,[y,y,y,y,y,y,y,y]], ...]
    # the id hereby should be the index of the entry
    for entry in library:
        if checkIfArraysAreEqualWoOrder(array, entry[1]):
            return entry[0]
    
    print("No array found. -1 will be returned")
    return -1
            
    # go over entries in array and compare/check if they are in the entries (of entries) of the library
    # select the entry from the library that contains most entries from the array as best choice
    # return its ID
    

def getMostSimilarArrayIDsWoOrder(array, library, maxDiffVal):
    
    arrayIDs = []
    for i in range(0, len(library)):
        if getNumberOfDiffElementsWoOrder(array, library[i][1]) < maxDiffVal:
            arrayIDs.append(i)
    return arrayIDs
    
    
    return arrayID
    
def checkIfArraysAreEqualWoOrder(array1, array2):
    if len(array1) != len(array2):
        return False
    else:
        return coll.Counter(array1) == coll.Counter(array2)

    
def getSimilarityOfArrays(array1, array2):
    if len(array1) < len(array2):
        getNumberDiffElementsOfOrderedArrays(array1, array2)
    

def getNumberOfDiffElementsWoOrder(array1, array2):
    if len(array1) < len(array2):
        return list(coll.Counter(array2) - coll.Counter(array1))
    else:
        return list(coll.Counter(array1) - coll.Counter(array2))

        

    
def getNumberDiffElementsOfOrderedArrays(smallArray, largeArray):
    print("Not tested yet")
    diffElements = 0
    for i in range(0, len(smallArray)):
        if i + diffElements >= len(largeArray):
            break
            
        if smallArray[i] == largeArray[i + diffElements]:
            continue
        else:
            diffElements += 1
    
    return diffElements
    
    
    
def getShiftsOfArray(array, targetArray):
    shifts = 0
    
    # shift the array until it matches the targetArray
    for i in range(0, len(array)):
        if (np.array(array) == np.array(targetArray)).all():
            break
        #count the shifts
        
        shifts += 1
        array = np.roll(array, 1)
    
    #return shifts
    return shifts







def getSortedCountAndCenterArray(groupedArray, outlineID, contours):
    countArray = []
    angleArray = []
    centerArray = []
    
    if outlineID > len(contours):
        return []
    
    outlineX, outlineY = getCenterCoordinates([outlineID], contours)
    
    
    for group in groupedArray:
        dotsX, dotsY = getCenterCoordinates(group, contours)
        index, angleArray = getSortedAngleArrayAndInsertID(outlineX, outlineY, dotsX, dotsY, angleArray)
        countArray.insert(index, len(group))
        centerArray.insert(index, [dotsX, dotsY])
    print("angle array ", angleArray)
    print("sorted count array ", countArray)
    print("sorted center array ", centerArray)
    return countArray, centerArray

# this will return the id and the sorted array after cosine values
def getSortedAngleArrayAndInsertID(outlineX, outlineY, dotsX, dotsY, angleArray):
    
    vec = [dotsX - outlineX, dotsY - outlineY] #cv2.normalize(dotsC) - cv2.normalize(outlineC)
    
    angle = getAngle(vec)
    
    if len(angleArray) < 1:
        angleArray.append(angle)
        return 0, angleArray
    else:
        for i in range(0, len(angleArray)):
            if angle < angleArray[i]:
                angleArray.insert(i, angle)
                return i, angleArray
        angleArray.append(angle)
        return len(angleArray) - 1, angleArray


def getAngle(vec):
    #print("Vector before norm ", vec)
    #norm = np.linalg.norm(vec)
    #vec = vec / norm
    #print("Vector ", vec)
    arctan = np.arctan2(vec[1], vec[0])
    angle = np.degrees(arctan) % 360
    #print("Angle ", angle)
    return angle
        
def getCenterCoordinates(ids, contours):
    if len(ids) > 0:
        cX = 0
        cY = 0

        for ID in ids:
            M = cv2.moments(contours[ID])
            if float(M["m00"]) > 0:
                x = float(M["m10"] / M["m00"])
                y = float(M["m01"] / M["m00"])
                cX = cX + x
                cY = cY + y
            
        cX = int(cX / len(ids))
        cY = int(cY / len(ids))
        
        return cX, cY

    
def createContourTrackbar():
    # creating trackbar to change contour size
    cv2.namedWindow('Contour Track Bar', cv2.WINDOW_NORMAL)
    surPix = 'SurPix'
    subtrPix = 'SubtrPix'
    minSizeDots = 'MinDot'
    maxSizeDots = 'MaxDot'
    minEcc = 'MinEcc'
    maxEcc = 'MaxEcc'
    minSizeOutline = 'MinOutline'
    maxSizeOutline = 'MaxOutline'
    maxDist = 'MaxDist'
    angleRot = 'Rot'
    
    
    bar = 'SizeBars'
    cv2.createTrackbar("SurPix", "Contour Track Bar", 5, 100, trackChanged)
    cv2.createTrackbar("SubtrPix", "Contour Track Bar", 2, 100, trackChanged)
    cv2.createTrackbar("MaxDot", "Contour Track Bar", 0, 1000, trackChanged)
    cv2.createTrackbar("MinDot", "Contour Track Bar", 0, 50, trackChanged)
    cv2.createTrackbar("MinEcc", "Contour Track Bar", 0, 100, trackChanged)
    cv2.createTrackbar("MaxEcc", "Contour Track Bar", 0, 100, trackChanged)
    cv2.createTrackbar("MaxOutline", "Contour Track Bar", 0, 20000, trackChanged)
    cv2.createTrackbar("MinOutline", "Contour Track Bar", 0, 5000, trackChanged)
    cv2.createTrackbar("MaxDist", "Contour Track Bar", 0, 100, trackChanged)
    cv2.createTrackbar("Rot", "Contour Track Bar", 0, 359, trackChanged)

def createImageTrackbar():
    # creating trackbar to change contour size
    cv2.namedWindow('Image Track Bar', cv2.WINDOW_NORMAL)
    roi_x = 'roi_x'
    roi_y = 'roi_y'
    
    
    bar = 'ImageBars'
    cv2.createTrackbar("roi_x", "Image Track Bar", 0, 1000, trackChanged)
    cv2.createTrackbar("roi_y", "Image Track Bar", 0, 1000, trackChanged)
    
    
    
def pointTracking(camImg):
    
    
    
    # reading image
    img = camImg
    imgGray = GetBlurredGrayScaleImageFromImage(img)
#     img = cv2.imread('SquareRotated2.jpg')
#     img = cv2.resize(img, (800, 800))
#     imgGray = GetBlurredGrayScaleImage('SquareRotated2.jpg')
    
#     img = cv2.imread('SquareWithPointCode.jpg')
#     img = cv2.resize(img, (800, 800))
#     imgGray = GetBlurredGrayScaleImage('SquareWithPointCode.jpg')
    
    
    


    # absolute thresholding
    #_, thrash = cv2.threshold(imgGray, 45, 255, cv2.THRESH_BINARY_INV)

    # adaptive thresholding
    surrPixel = cv2.getTrackbarPos("SurPix", "Contour Track Bar")
    # surrounding pixel must be odd number
    if surrPixel % 2 == 0:
        surrPixel += 1
    subtrPixel = cv2.getTrackbarPos("SubtrPix", "Contour Track Bar")
    # mean thresholding
    # adaptiveThreshold(src, dst, 125, Imgproc.ADAPTIVE_THRESH_MEAN_C, Imgproc.THRESH_BINARY, number_of_surrounding_pixels(must be odd), 12)
    adapMeanThresh = cv2.adaptiveThreshold(imgGray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, surrPixel, subtrPixel)
    # gaussian thresholding
    adapGausThresh = cv2.adaptiveThreshold(imgGray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, surrPixel, subtrPixel)

    
    
    # library for TEST purpose
    # get later this library from external file and load it in once in the beginning
    idLibrary = [[0,[1, 4, 2, 1, 3, 2, 1, 3]], [1,[1,2,3,4,1,2,3,4]], [2,[1,1,1,1,1,1,1,1]]]

    # get track bar values
    minAreaDot = cv2.getTrackbarPos("MinDot", "Contour Track Bar")
    maxAreaDot = cv2.getTrackbarPos("MaxDot", "Contour Track Bar")
    minEcc = cv2.getTrackbarPos("MinEcc", "Contour Track Bar") * 0.01
    maxEcc = cv2.getTrackbarPos("MaxEcc", "Contour Track Bar") * 0.01
    minAreaOutline = cv2.getTrackbarPos("MinOutline", "Contour Track Bar")
    maxAreaOutline = cv2.getTrackbarPos("MaxOutline", "Contour Track Bar")
    maxDist = float(cv2.getTrackbarPos("MaxDist", "Contour Track Bar"))
    angle = float(cv2.getTrackbarPos("Rot", "Contour Track Bar"))

    
    # detect contour on threshold image
    detImg = adapGausThresh.copy()
    detImg = rotate_image(detImg, angle)

    # grab longest contour
    edged = cv2.Canny(detImg, 25, 325)
    contours, hierarchy = cv2.findContours(detImg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    hierarchy = hierarchy[0] # get the actual inner list of hierarchy descriptions
    #print("Hierarchy", hierarchy)


    # this should later be changed for a list of outlines of the tiles
    #outlineID = cv2.getTrackbarPos("OutlineID", "Contour Track Bar")

    contourImage = img.copy()
    contourImage = rotate_image(contourImage, angle)


    # get contours of certain area

    # get dot contours
    dotCon, dotHier = getContoursWithArea(contours, hierarchy, minAreaDot, maxAreaDot)
    #get outline contours
    outlineConIDs, outlineHier = getContourIDsWithArea(contours, hierarchy, minAreaOutline, maxAreaOutline)
    alreadyVisited = []
    for outlineID in outlineConIDs:
        #if the outline was already visited from another outline skip it
        if outlineID in alreadyVisited:
            continue

        # get dots of only one outline
        #singleTileDotCon = getDotsInOutline(outlineID, contours, hierarchy, minAreaDot, maxAreaDot)
        #singleTileDotIDs, alreadyVisitedIDs = getDotIDsInOutline(outlineID, contours, hierarchy, minAreaDot, maxAreaDot)
        singleTileDotIDs, alreadyVisitedIDs = getDotIDsInOutlineWithEccentricity(outlineID, contours, hierarchy, minAreaDot, maxAreaDot,minEcc, maxEcc)

        alreadyVisited.extend(alreadyVisitedIDs)
        #print("Already visited ", alreadyVisited)
        #print("single dots ids ", singleTileDotIDs)

        # get only inner most nodes for dots
        #dotCon, dotHier = getChildContours(dotCon, dotHier)




        # draw outlines

        # draw dots
        #DrawContours(dotCon, contourImage)
        # draw outlines
        DrawContoursFromID([outlineID], contours, contourImage, (255, 0, 0))
        if len(singleTileDotIDs) > 0:
            print("")
            #DrawContoursWithText(singleTileDotCon, contourImage, str(outlineID))
            groupedDots = getGroupedDotsIDs(singleTileDotIDs, contours, hierarchy, maxDist)
            DrawGroupedContoursFromID(groupedDots, contours, contourImage)
            countArray, centerArray = getSortedCountAndCenterArray(groupedDots, outlineID, contours)
            tileID, shifts = getTileIDAndShiftsFromGroupedArray(countArray, idLibrary)
            exactRot = getZRotationFromShifts(shifts, centerArray, outlineID, contours)
            xRot, yRot = getXYRotationFromShiftsForSquare(shifts, centerArray)
#             print("ID ", tileID)
#             print("shifts ", shifts)
            print("Z rotation ", exactRot)
            print("XY rotation ", str(xRot), str(yRot))
            DrawContoursWithTextInCenter([outlineID], contours, contourImage, str(exactRot), (0, 255, 0))
            #print("Grouped Array", countArray)
            #DrawContoursFromID(singleTileDotIDs, contours, contourImage)
        
        
    
    cv2.imshow("DetectionImage", detImg)
    cv2.imshow("Contour Image", contourImage)
    cv2.imshow("edged", edged)
        
        
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    
    
## Fast cam acquisition and MediaPipe POSE recognition and gesture recognition


# coding=utf-8
# =============================================================================
# Copyright (c) 2001-2021 FLIR Systems, Inc. All Rights Reserved.
#
# This software is the confidential and proprietary information of FLIR
# Integrated Imaging Solutions, Inc. ("Confidential Information"). You
# shall not disclose such Confidential Information and shall use it only in
# accordance with the terms of the license agreement you entered into
# with FLIR Integrated Imaging Solutions, Inc. (FLIR).
#
# FLIR MAKES NO REPRESENTATIONS OR WARRANTIES ABOUT THE SUITABILITY OF THE
# SOFTWARE, EITHER EXPRESSED OR IMPLIED, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR
# PURPOSE, OR NON-INFRINGEMENT. FLIR SHALL NOT BE LIABLE FOR ANY DAMAGES
# SUFFERED BY LICENSEE AS A RESULT OF USING, MODIFYING OR DISTRIBUTING
# THIS SOFTWARE OR ITS DERIVATIVES.
# =============================================================================
#
# This AcquireAndDisplay.py shows how to get the image data, and then display images in a GUI.
# This example relies on information provided in the ImageChannelStatistics.py example.
#
# This example demonstrates how to display images represented as numpy arrays.
# Currently, this program is limited to single camera use.
# NOTE: keyboard and matplotlib must be installed on Python interpreter prior to running this example.







global continue_recording
continue_recording = True

BG_COLOR = (0, 0, 0) # black
ROI_y = 650
ROI_x = 150
ROI_height = 800
ROI_width = 800


def handle_close(evt):
    """
    This function will close the GUI when close event happens.

    :param evt: Event that occurs when the figure closes.
    :type evt: Event
    """

    global continue_recording
    continue_recording = False


def acquire_and_display_images(cam, nodemap, nodemap_tldevice):
    """
    This function continuously acquires images from a device and display them in a GUI.

    :param cam: Camera to acquire images from.
    :param nodemap: Device nodemap.
    :param nodemap_tldevice: Transport layer device nodemap.
    :type cam: CameraPtr
    :type nodemap: INodeMap
    :type nodemap_tldevice: INodeMap
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    global continue_recording

    sNodemap = cam.GetTLStreamNodeMap()

    # Change bufferhandling mode to NewestOnly
    node_bufferhandling_mode = PySpin.CEnumerationPtr(sNodemap.GetNode('StreamBufferHandlingMode'))
    if not PySpin.IsAvailable(node_bufferhandling_mode) or not PySpin.IsWritable(node_bufferhandling_mode):
        print('Unable to set stream buffer handling mode.. Aborting...')
        return False

    # Retrieve entry node from enumeration node
    node_newestonly = node_bufferhandling_mode.GetEntryByName('NewestOnly')
    if not PySpin.IsAvailable(node_newestonly) or not PySpin.IsReadable(node_newestonly):
        print('Unable to set stream buffer handling mode.. Aborting...')
        return False

    # Retrieve integer value from entry node
    node_newestonly_mode = node_newestonly.GetValue()

    # Set integer value from entry node as new value of enumeration node
    node_bufferhandling_mode.SetIntValue(node_newestonly_mode)

    print('*** IMAGE ACQUISITION ***\n')
    try:
        
        
        ## media pipe?
        node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        if not PySpin.IsAvailable(node_acquisition_mode) or not PySpin.IsWritable(node_acquisition_mode):
            print('Unable to set acquisition mode to continuous (enum retrieval). Aborting...')
            return False

        # Retrieve entry node from enumeration node
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        if not PySpin.IsAvailable(node_acquisition_mode_continuous) or not PySpin.IsReadable(
                node_acquisition_mode_continuous):
            print('Unable to set acquisition mode to continuous (entry retrieval). Aborting...')
            return False

        # Retrieve integer value from entry node
        acquisition_mode_continuous = node_acquisition_mode_continuous.GetValue()

        # Set integer value from entry node as new value of enumeration node
        node_acquisition_mode.SetIntValue(acquisition_mode_continuous)

        print('Acquisition mode set to continuous...')

        #  Begin acquiring images
        #
        #  *** NOTES ***
        #  What happens when the camera begins acquiring images depends on the
        #  acquisition mode. Single frame captures only a single image, multi
        #  frame catures a set number of images, and continuous captures a
        #  continuous stream of images.
        #
        #  *** LATER ***
        #  Image acquisition must be ended when no more images are needed.
        cam.BeginAcquisition()

        print('Acquiring images...')

        #  Retrieve device serial number for filename
        #
        #  *** NOTES ***
        #  The device serial number is retrieved in order to keep cameras from
        #  overwriting one another. Grabbing image IDs could also accomplish
        #  this.
        device_serial_number = ''
        node_device_serial_number = PySpin.CStringPtr(nodemap_tldevice.GetNode('DeviceSerialNumber'))
        if PySpin.IsAvailable(node_device_serial_number) and PySpin.IsReadable(node_device_serial_number):
            device_serial_number = node_device_serial_number.GetValue()
            print('Device serial number retrieved as %s...' % device_serial_number)

        # Close program
        print('Press enter to close the program..')

        # Figure(1) is default so you can omit this line. Figure(0) will create a new window every time program hits this line
        fig = plt.figure(1)

        # Close the GUI when close event happens
        fig.canvas.mpl_connect('close_event', handle_close)

    

        createContourTrackbar()
        createImageTrackbar()
        
        
        while(continue_recording):
            try:

                # Process Key (ESC: end) #################################################
                key = cv2.waitKey(1)
                if key == 27:  # ESC
                    break

                #  Retrieve next received image
                #
                #  *** NOTES ***
                #  Capturing an image houses images on the camera buffer. Trying
                #  to capture an image that does not exist will hang the camera.
                #
                #  *** LATER ***
                #  Once an image from the buffer is saved and/or no longer
                #  needed, the image must be released in order to keep the
                #  buffer from filling up.

                image_result = cam.GetNextImage(1000)

                #  Ensure image completion
                if image_result.IsIncomplete():
                    print('Image incomplete with image status %d ...' % image_result.GetImageStatus())

                else:

                    ## This is 2048x2048 so we need to crop it to the ROI from the lense
                    width = image_result.GetWidth()
                    height = image_result.GetHeight()


                    ## This converts it to GreyScale
                    #image_converted = image_result.Convert(spin.PixelFormat_Mono8, spin.HQ_LINEAR)
                    ## This converts it to RGB
                    image_converted = image_result.Convert(PySpin.PixelFormat_BGR8)
                    rgb_array = image_converted.GetData()
                    rgb_array = rgb_array.reshape(height, width, 3)


                    ## process mediapipe on image
                    #image_rgb = cv2.cvtColor(cv2.flip(rgb_array, 1), cv2.COLOR_BGR2RGB)
                    image_rgb = cv2.flip(rgb_array, 1)


                    ## **** Resizing / Croping *****

                    ## RESIZING the image since it would be 2048x2048 otherwise (kind of too big for the window)

                    ## we might have to size this further down for mediapipe to run fast
                    #image_rgb = cv2.resize(image_rgb, (800, 800))
                    # scale = 800/2048   
                    ## this is to display potential cropping region in the downscaled image
                    # cv2.rectangle(image_rgb, (int(650*scale), int(580*scale)), (int(1450*scale), int(1380*scale)), (0, 255, 0), 3)

                    ## CROPPPING the region of the lense (should be around 800 to fit with the setup so far...)
                    ## This is the cropping
                    ## These values are taken from the unity config
                    ## They might change...
                    ROI_y = cv2.getTrackbarPos("roi_y", "Image Track Bar")
                    ROI_x = cv2.getTrackbarPos("roi_x", "Image Track Bar")
                    array_cropped = image_rgb[ROI_y:(ROI_y + ROI_height), ROI_x:(ROI_x + ROI_width)]
                    image_rgb = array_cropped.copy() # needed to get the correct data format for further processing




                    ## **** un-distort image

                    #undistorted_img = cv2.remap(image_rgb, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

                    ## media pipe works better on rgb image
                    image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB)
                    image_rgb = cv2.resize(image_rgb, (800, 800))


                    
                    ## do stuff here
                    pointTracking(image_rgb)
                    


                    ## convert image back to bgr for outputting it in rgb with the cam (python is confusing...)
                    image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)










                    #cv2.imshow('Camera Image', image_rgb) 
                    if cv2.waitKey(5) & 0xFF == 27:
                        continue_recording=False    
                        break



                # If user presses enter, close the program
                if keyboard.is_pressed('ENTER'):
                    print('Program is closing...')

                    # Close figure
                    plt.close('all')             
                    input('Done! Press Enter to exit...')
                    continue_recording=False                        

                #  Release image
                #
                #  *** NOTES ***
                #  Images retrieved directly from the camera (i.e. non-converted
                #  images) need to be released in order to keep from filling the
                #  buffer.
                image_result.Release()

            except PySpin.SpinnakerException as ex:
                print('Error: %s' % ex)
                return False

        #  End acquisition
        #
        #  *** NOTES ***
        #  Ending acquisition appropriately helps ensure that devices clean up
        #  properly and do not need to be power-cycled to maintain integrity.
        print("Ended acquisition")
        cam.EndAcquisition()
        cv2.destroyAllWindows()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True







def logging_csv(number, mode, landmark_list, point_history_list):
    if mode == 0:
        pass
    if mode == 1 and (0 <= number <= 9):
        csv_path = 'model/keypoint_classifier/keypoint.csv'
        with open(csv_path, 'a', newline="") as f:
            writer = csv.writer(f)
            writer.writerow([number, *landmark_list])
    if mode == 2 and (0 <= number <= 9):
        csv_path = 'model/point_history_classifier/point_history.csv'
        with open(csv_path, 'a', newline="") as f:
            writer = csv.writer(f)
            writer.writerow([number, *point_history_list])
    return






















def run_single_camera(cam):
    """
    This function acts as the body of the example; please see NodeMapInfo example
    for more in-depth comments on setting up cameras.

    :param cam: Camera to run on.
    :type cam: CameraPtr
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    try:
        result = True

        nodemap_tldevice = cam.GetTLDeviceNodeMap()

        # Initialize camera
        cam.Init()

        # Retrieve GenICam nodemap
        nodemap = cam.GetNodeMap()

        # Acquire images
        result &= acquire_and_display_images(cam, nodemap, nodemap_tldevice)

        # Deinitialize camera
        cam.DeInit()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result


def main():
    """
    Example entry point; notice the volume of data that the logging event handler
    prints out on debug despite the fact that very little really happens in this
    example. Because of this, it may be better to have the logger set to lower
    level in order to provide a more concise, focused log.

    :return: True if successful, False otherwise.
    :rtype: bool
    """
    result = True

    # Retrieve singleton reference to system object
    system = PySpin.System.GetInstance()

    # Get current library version
    version = system.GetLibraryVersion()
    print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))

    # Retrieve list of cameras from the system
    cam_list = system.GetCameras()

    num_cameras = cam_list.GetSize()

    print('Number of cameras detected: %d' % num_cameras)

    # Finish if there are no cameras
    if num_cameras == 0:

        # Clear camera list before releasing system
        cam_list.Clear()

        # Release system instance
        system.ReleaseInstance()

        print('Not enough cameras!')
        input('Done! Press Enter to exit...')
        return False

    # Run example on each camera
    for i, cam in enumerate(cam_list):

        print('Running example for camera %d...' % i)

        result &= run_single_camera(cam)
        print('Camera %d example complete... \n' % i)

    # Release reference to camera
    # NOTE: Unlike the C++ examples, we cannot rely on pointer objects being automatically
    # cleaned up when going out of scope.
    # The usage of del is preferred to assigning the variable to None.
    del cam

    # Clear camera list before releasing system
    cam_list.Clear()

    # Release system instance
    system.ReleaseInstance()

    input('Done! Press Enter to exit...')
    cv2.destroyAllWindows()
    return result


if __name__ == '__main__':
    if main():
        sys.exit(0)
    else:
        sys.exit(1)


Library version: 2.4.0.144
Number of cameras detected: 1
Running example for camera 0...
*** IMAGE ACQUISITION ***

Acquisition mode set to continuous...
Acquiring images...
Device serial number retrieved as 20413825...
Press enter to close the program..

angle array  [0.0]
sorted count array  [1]
sorted center array  [[195, 271]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [288.434948822922]
sorted count array  [1]
sorted center array  [[378, 481]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [86.63353933657021]
sorted count array  [1]
sorted center array  [[379, 500]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [273.5763343749974]
sorted count array  [1]



angle array  [273.5763343749974]
sorted count array  [1]
sorted center array  [[380, 448]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [73.30075576600639]
sorted count array  [1]
sorted center array  [[377, 403]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [69.44395478041653]
sorted count array  [1]
sorted center array  [[377, 403]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [66.80140948635182]
sorted count array  [1]
sorted center array  [[377, 403]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [68.19859051364818]
sorted count array  [1]
sorted


angle array  [0.0]
sorted count array  [1]
sorted center array  [[483, 273]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [251.56505117707798]
sorted count array  [1]
sorted center array  [[370, 475]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [270.0]
sorted count array  [1]
sorted center array  [[378, 481]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [93.09405805891711]
sorted count array  [1]
sorted center array  [[371, 374]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [78.69006752597979]
sorted count array  [1]
sorted center array  [[379, 503


angle array  [171.86989764584402]
sorted count array  [1]
sorted center array  [[336, 411]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [142.59464336859145]
sorted count array  [1]
sorted center array  [[338, 432]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [312.35745470593525]
sorted count array  [1]
sorted center array  [[403, 371]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [123.69006752597979]
sorted count array  [1]
sorted center array  [[371, 496]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [268.492564241225]
sorted count array  [1]
sor


angle array  [338.1985905136482]
sorted count array  [1]
sorted center array  [[409, 338]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [32.47119229084849]
sorted count array  [1]
sorted center array  [[405, 365]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [0.0]
sorted count array  [1]
sorted center array  [[413, 392]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [41.98721249581666]
sorted count array  [1]
sorted center array  [[402, 372]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [32.47119229084849]
sorted count array  [1]
sorted center array 

...


angle array  [20.136303428248137, 33.690067525979785, 45.0, 239.74356283647074, 246.80140948635182, 274.3987053549955, 304.695153531234, 335.55604521958344, 352.8749836510982]
sorted count array  [1, 1, 1, 1, 1, 1, 1, 1, 1]
sorted center array  [[547, 385], [526, 380], [541, 398], [496, 338], [514, 367], [519, 348], [544, 335], [528, 369], [549, 370]]
No array found. -1 will be returned
Need to test with rotated picture
ratio  5.4
Need to test with rotated picture
ratio  0.26666666666666666
Z rotation  -1
XY rotation  90.0 -66.0

angle array  [0.0]
sorted count array  [1]
sorted center array  [[547, 385]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [65.4925947698703]
sorted count array  [1]
sorted center array  [[407, 439]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle arra


angle array  [45.0, 46.12330271407543, 235.56101069119637, 304.695153531234, 321.3401917459099, 354.4724598483438]
sorted count array  [1, 1, 1, 1, 3, 1]
sorted center array  [[528, 378], [545, 396], [496, 335], [547, 331], [525, 366], [551, 367]]
No array found. -1 will be returned
Need to test with rotated picture
ratio  0.9062499999999999
Need to test with rotated picture
Z rotation  -1
XY rotation  -8.437500000000014 None

angle array  [0.0]
sorted count array  [1]
sorted center array  [[449, 459]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [21.161259816828277, 38.65980825409009, 46.12330271407543, 234.78240703180728, 236.30993247402023, 304.695153531234, 341.565051177078, 354.6441749571448]
sorted count array  [1, 1, 1, 1, 1, 1, 1, 1]
sorted center array  [[551, 382], [530, 378], [545, 396], [496, 336], [516, 364], [547, 331], [532, 366], [552, 367]]
zero angle  21.161


angle array  [38.15722658736906, 242.48799737614854, 247.06789956241022, 249.44395478041653, 318.01278750418334, 343.3007557660064]
sorted count array  [1, 1, 2, 1, 1, 1]
sorted center array  [[539, 363], [486, 293], [500, 315], [508, 333], [521, 332], [541, 332]]
No array found. -1 will be returned
Need to test with rotated picture
ratio  0.5384615384615384
Need to test with rotated picture
Z rotation  -1
XY rotation  -41.53846153846154 None

angle array  [34.5085229876684, 240.25511870305778, 241.55707137563667, 245.09523119190482, 327.8750015596125, 329.74356283647074, 343.3007557660064]
sorted count array  [2, 1, 1, 3, 1, 3, 1]
sorted center array  [[542, 361], [506, 332], [484, 291], [497, 311], [553, 312], [522, 332], [540, 330]]
No array found. -1 will be returned
Need to test with rotated picture
ratio  2.0294117647058827
Need to test with rotated picture
ratio  1.6923076923076923
Z rotation  -1
XY rotation  23.161764705882362 15.576923076923077

angle array  [0.0]
sorted coun


angle array  [235.0079798014413]
sorted count array  [1]
sorted center array  [[367, 480]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [15.945395900922854, 296.565051177078, 317.9108378261678]
sorted count array  [1, 1, 2]
sorted center array  [[505, 383], [491, 331], [501, 345]]
No array found. -1 will be returned
Need to test with rotated picture
ratio  2.5
Need to test with rotated picture
Z rotation  -1
XY rotation  33.75 None

angle array  [244.98310652189997]
sorted count array  [1]
sorted center array  [[364, 478]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with rotated picture
Z rotation  -1
XY rotation  None None

angle array  [15.124007308310569, 261.7028550301631]
sorted count array  [1, 1]
sorted center array  [[513, 369], [469, 311]]
No array found. -1 will be returned
Need to test with rotated picture
Need to test with ro


angle array  [144.83356396420712, 249.7174409110834, 270.0, 295.2531633945739, 306.86989764584405, 320.82634202955575, 331.99082329198615, 342.4744316262771, 356.0995062576181]
sorted count array  [1, 1, 1, 1, 1, 1, 1, 1, 1]
sorted center array  [[397, 334], [424, 257], [441, 255], [466, 250], [453, 287], [468, 281], [488, 278], [460, 297], [485, 300]]
No array found. -1 will be returned
Need to test with rotated picture
ratio  4.0
Need to test with rotated picture
ratio  1.2571428571428571
Z rotation  -1
XY rotation  67.5 5.785714285714285

angle array  [145.0975043840753, 250.42687416958978, 270.0, 318.73139699916044, 324.7824070318073, 356.00908690157024]
sorted count array  [1, 1, 1, 1, 3, 1]
sorted center array  [[397, 333], [424, 258], [440, 256], [489, 260], [474, 279], [483, 300]]
No array found. -1 will be returned
Need to test with rotated picture
ratio  0.7906976744186046
Need to test with rotated picture
Z rotation  -1
XY rotation  -18.83720930232559 None

angle array  [14